# 本コードについて
[pandas](https://github.com/pandas-dev/pandas)ライブラリを使ってユーザ一覧、ユーザ操作一覧、リアクション一覧を集計するサンプル

幾つかの例を踏まえつつ、ユーザごとの活動度(actions.point + reactions.reward)のランキングを出すことを目標とする

## データの関係
```
< users > (親)
| --- < actions > (子) 
  | --- < reactions > (孫)
```

## データ構造
- users.csv: ユーザ一覧
  - id: ユーザID。常にユニークな文字列
  - name: ユーザ名
  - email: ユーザのemailアドレス
  - type: ユーザの種類。personalは個人で、enterpriseは営利団体としている
  - leaved: 退会済みかどうか
- actions.csv: ユーザ操作一覧
  - id: アクションID。常にユニークな文字列
  - user_id: ユーザ操作をしたユーザのID
  - date: 操作日時 ※
  - message: 操作に関するメッセージ
  - point: 操作により得られたポイント。整数値 or 欠損値
- reactions.csv: ユーザ操作に対するリアクション一覧
  - id: リアクションID
  - action_id: リアクション対象のアクションID
  - user_id: リアクション元のユーザID
  - date: リアクション日時 ※
  - type: リアクションの種類
  - reward: リアクションによる受け取りポイント
  
※ いずれもISO8601フォーマットの文字列とする

In [1]:
import numpy as np
import pandas as pd

# ユーザデータの解析

In [2]:
df_users = pd.read_csv("users.csv")
df_users

,id,name,email,type,leaved
0,BcHgeZkTsc,アリス,hoge@hoge.example.com,personal,False
1,KHPiabVr3o,ボブ,fuga@fuga.org,personal,False
2,AQA7LkexXv,チャーリー,foo@foo.net,personal,True
3,C82ZQKSQk7,ダービー,baz@baz.biz,enterprise,False
4,HzZow64HGH,エリザベス,bar@bar.info,enterprise,False


## idとnameを取り出す
以下の様に、listで必要な列を指定するか、locで範囲指定する

- listの場合、列が隣り合ってなくてもよい
- locの場合、iloc(indexによる範囲指定)と異なり終端も含む

In [3]:
print(df_users[["id", "name"]])
print(df_users.loc[:, :"name"])

           id   name
0  BcHgeZkTsc    アリス
1  KHPiabVr3o     ボブ
2  AQA7LkexXv  チャーリー
3  C82ZQKSQk7   ダービー
4  HzZow64HGH  エリザベス
           id   name
0  BcHgeZkTsc    アリス
1  KHPiabVr3o     ボブ
2  AQA7LkexXv  チャーリー
3  C82ZQKSQk7   ダービー
4  HzZow64HGH  エリザベス


## enterprise ユーザ or 退会済みユーザを取り出す: query等
queryの場合、条件を文字列で渡す

複数条件指定時、ANDは"&"、ORは"|"でつなぐ

条件式は必ず()で括らないと意図した評価順にならず、`TypeError`となる

In [4]:
df_users.query('type == "enterprise" or leaved == True')

,id,name,email,type,leaved
2,AQA7LkexXv,チャーリー,foo@foo.net,personal,True
3,C82ZQKSQk7,ダービー,baz@baz.biz,enterprise,False
4,HzZow64HGH,エリザベス,bar@bar.info,enterprise,False


In [5]:
df_users[
    (df_users.type == "enterprise")
    | (df_users.leaved) == True]

,id,name,email,type,leaved
2,AQA7LkexXv,チャーリー,foo@foo.net,personal,True
3,C82ZQKSQk7,ダービー,baz@baz.biz,enterprise,False
4,HzZow64HGH,エリザベス,bar@bar.info,enterprise,False


## ユーザ数を出す

### 全ユーザ数

In [6]:
len(df_users)

5

### leavedがFalseであるユーザの数(入会ユーザ数)

In [7]:
len(df_users[df_users.leaved == False])

4

### メールアドレスが"b"から始まっているユーザを取りだし: map, (apply)

今回のケースでは、df_usersのemailだけ参照すればいいのでmapを使う方法が望ましい(よりスループットが高く、必要なデータのみ触るため)

反対に、各行の複数パラメータを参照したい場合はapplyを使う


In [8]:
serial_email_b_start = df_users.email.map(lambda e: e.startswith("b"))
print(len(df_users[serial_email_b_start]))

serial_email_b_start = df_users.apply(
    lambda row: row.email.startswith("b"), axis=1)
print(len(df_users[serial_email_b_start]))

2
2


## ユニークなユーザタイプをリストアップし、カウントアップ: drop_duplicates, to_list

In [9]:
user_types = df_users.type.drop_duplicates().to_list()
user_types

['personal', 'enterprise']

In [10]:
{
    user_type: len(df_users[df_users.type == user_type])
    for user_type in user_types
} 

{'personal': 3, 'enterprise': 2}

# ユーザによる操作データの解析

id="OoXNK4b3px"について、actions.csv内でpointが空なのでNaNになる

NaNはfloat型なので、暗黙の型変換でpointはfloat型になる

In [11]:
df_actions = pd.read_csv("actions.csv")
df_actions

,id,user_id,date,message,point
0,7GsubTX9n6,BcHgeZkTsc,2021-08-15T10:12:34+09:00,Hoge hoge,1.0
1,D76FJVQ2j2,KHPiabVr3o,2021-08-15T10:23:45+09:00,Lorem Ipsum,21.0
2,6znyhCukd6,HzZow64HGH,2021-08-15T11:54:32+09:00,テスト テスト,3.0
3,hSQszmDjlU,BcHgeZkTsc,2021-08-15T12:34:56+09:00,テスト\nテスト２,42.0
4,CVqQD0xH2Y,HzZow64HGH,2021-08-15T14:36:52+09:00,👑👑💢,49.0
5,OoXNK4b3px,BcHgeZkTsc,2021-08-15T14:41:03+09:00,🔡,NaN
6,veDQHBOXnG,BcHgeZkTsc,2021-08-15T14:52:12+09:00,foo,4.0


## 投稿データをuser_id昇順→point降順にする: sort_values

引数で渡すリストの各アイテムの順番がそれぞれ対応する(以下例であれば、user_idがascending=True, pointがascending=False)

reset_indexすることにより、indexを0から振り直している(drop=Trueにより、reset前のindexが列に追加されないようにしている)

In [12]:
df_actions \
    .sort_values(["user_id", "point"], ascending=[True, False]) \
    .reset_index(drop=True)

,id,user_id,date,message,point
0,hSQszmDjlU,BcHgeZkTsc,2021-08-15T12:34:56+09:00,テスト\nテスト２,42.0
1,veDQHBOXnG,BcHgeZkTsc,2021-08-15T14:52:12+09:00,foo,4.0
2,7GsubTX9n6,BcHgeZkTsc,2021-08-15T10:12:34+09:00,Hoge hoge,1.0
3,OoXNK4b3px,BcHgeZkTsc,2021-08-15T14:41:03+09:00,🔡,NaN
4,CVqQD0xH2Y,HzZow64HGH,2021-08-15T14:36:52+09:00,👑👑💢,49.0
5,6znyhCukd6,HzZow64HGH,2021-08-15T11:54:32+09:00,テスト テスト,3.0
6,D76FJVQ2j2,KHPiabVr3o,2021-08-15T10:23:45+09:00,Lorem Ipsum,21.0


## pointの値なし(NaN)を0埋めし、int型にする: fillna, astype
Pythonのint型指定時、[64bitの符号付き整数](https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.int_)に変換される

In [13]:
df_actions.point = df_actions.point.fillna(0).astype(int)
df_actions[["id", "point"]]

,id,point
0,7GsubTX9n6,1
1,D76FJVQ2j2,21
2,6znyhCukd6,3
3,hSQszmDjlU,42
4,CVqQD0xH2Y,49
5,OoXNK4b3px,0
6,veDQHBOXnG,4


## ユーザごとにpointを合算: groupby

groupbyの引数で指定した"user_id"はIndexに変換される

aggにて、どのパラメータをどの関数・処理で集計するかを指定する(指定しないパラメータは出力されない)

Indexに変換したくない場合、as_index=Falseを指定する(後述)

In [14]:
df_actions \
    .groupby(["user_id"]) \
    .agg({
        "point": sum,
    })

,point
user_id,
BcHgeZkTsc,47
HzZow64HGH,52
KHPiabVr3o,21


## ユーザごとのpointの統計値を出す: groupby, describe

In [15]:
df_actions \
    .groupby(["user_id"]) \
    .agg({
        "point": "describe",
    })

point                                                 
           count   mean        std   min    25%   50%   75%   max
user_id                                                          
BcHgeZkTsc   4.0  11.75  20.238165   0.0   0.75   2.5  13.5  42.0
HzZow64HGH   2.0  26.00  32.526912   3.0  14.50  26.0  37.5  49.0
KHPiabVr3o   1.0  21.00        NaN  21.0  21.00  21.0  21.0  21.0

## ユーザごとに初めてのaction取り出し: groupby

In [16]:
function_name = "first"
df_actions \
    .groupby(["user_id"], as_index=False) \
    .agg({
        "date": function_name,
        "message": function_name,
        "point": function_name,
    })

,user_id,date,message,point
0,BcHgeZkTsc,2021-08-15T10:12:34+09:00,Hoge hoge,1
1,HzZow64HGH,2021-08-15T11:54:32+09:00,テスト テスト,3
2,KHPiabVr3o,2021-08-15T10:23:45+09:00,Lorem Ipsum,21


## 10分ごとのpoint値の小計を出力: to_datetime, resample
resample + sumにより、10分ごとの区間でpointの小計を取る。

-5Minのoffsetを入れることで、最初action日時=2021-08-15T10:12:34+09:00を含む区間として10:05(=10:10 - 0:05)から集計していく


In [17]:
df_actions_date = df_actions.copy()
df_actions_date.index = pd.to_datetime(df_actions.date)
df_actions_date.resample("10Min", offset="-5Min").sum()

,point
date,
2021-08-15 10:05:00+09:00,1
2021-08-15 10:15:00+09:00,21
2021-08-15 10:25:00+09:00,0
2021-08-15 10:35:00+09:00,0
2021-08-15 10:45:00+09:00,0
2021-08-15 10:55:00+09:00,0
2021-08-15 11:05:00+09:00,0
2021-08-15 11:15:00+09:00,0
2021-08-15 11:25:00+09:00,0


## actionに対するリアクションデータの解析

In [18]:
df_reactions_1 = pd.read_csv("reactions_1.csv")
df_reactions_1

,id,action_id,user_id,date,type,reward
0,ycV0zbqrL5,7GsubTX9n6,AQA7LkexXv,2021-08-15T11:52:12+09:00,like,1
1,NXXk7iEsMA,OoXNK4b3px,AQA7LkexXv,2021-08-15T13:09:31+09:00,like,1
2,Oq9i6DBTBp,OoXNK4b3px,AQA7LkexXv,2021-08-15T13:21:47+09:00,like,1


In [19]:
df_reactions_2 = pd.read_csv("reactions_2.csv")
df_reactions_2

,id,action_id,user_id,date,type,reward
0,jwUcUCuVEq,D76FJVQ2j2,BcHgeZkTsc,2021-08-15T16:21:48+09:00,comment,3


## 2データを1つのDataFrameに結合する: concat

In [20]:
df_reactions = pd.concat([df_reactions_1, df_reactions_2])
df_reactions

,id,action_id,user_id,date,type,reward
0,ycV0zbqrL5,7GsubTX9n6,AQA7LkexXv,2021-08-15T11:52:12+09:00,like,1
1,NXXk7iEsMA,OoXNK4b3px,AQA7LkexXv,2021-08-15T13:09:31+09:00,like,1
2,Oq9i6DBTBp,OoXNK4b3px,AQA7LkexXv,2021-08-15T13:21:47+09:00,like,1
0,jwUcUCuVEq,D76FJVQ2j2,BcHgeZkTsc,2021-08-15T16:21:48+09:00,comment,3


# 複数データをmergeして解析する

## action, reactionデータをactionのidで左結合: merge, (rename)

In [21]:
df_reactions_renamed = df_reactions.rename(columns={
    "id": "reaction_id", 
    "user_id": "reaction_user_id",
    "date": "reaction_date",
})
df_actions_renamed = df_actions.rename(columns={"id": "action_id"})

df_actions_merged = df_actions_renamed.merge(
    df_reactions_renamed, on=["action_id"], how="left")

df_actions_merged.reward = \
    df_actions_merged.reward.fillna(0).astype(int)
df_actions_merged

,action_id,user_id,date,message,point,reaction_id,reaction_user_id,reaction_date,type,reward
0,7GsubTX9n6,BcHgeZkTsc,2021-08-15T10:12:34+09:00,Hoge hoge,1,ycV0zbqrL5,AQA7LkexXv,2021-08-15T11:52:12+09:00,like,1
1,D76FJVQ2j2,KHPiabVr3o,2021-08-15T10:23:45+09:00,Lorem Ipsum,21,jwUcUCuVEq,BcHgeZkTsc,2021-08-15T16:21:48+09:00,comment,3
2,6znyhCukd6,HzZow64HGH,2021-08-15T11:54:32+09:00,テスト テスト,3,NaN,NaN,NaN,NaN,0
3,hSQszmDjlU,BcHgeZkTsc,2021-08-15T12:34:56+09:00,テスト\nテスト２,42,NaN,NaN,NaN,NaN,0
4,CVqQD0xH2Y,HzZow64HGH,2021-08-15T14:36:52+09:00,👑👑💢,49,NaN,NaN,NaN,NaN,0
5,OoXNK4b3px,BcHgeZkTsc,2021-08-15T14:41:03+09:00,🔡,0,NXXk7iEsMA,AQA7LkexXv,2021-08-15T13:09:31+09:00,like,1
6,OoXNK4b3px,BcHgeZkTsc,2021-08-15T14:41:03+09:00,🔡,0,Oq9i6DBTBp,AQA7LkexXv,2021-08-15T13:21:47+09:00,like,1
7,veDQHBOXnG,BcHgeZkTsc,2021-08-15T14:52:12+09:00,foo,4,NaN,NaN,NaN,NaN,0


## 上記データにユーザ名を追加: merge
df_usersにあるemail, type, leavedは以後使わない

そのため、"id", "name"列以外をdropしている(下記コードの1行目)

df_actions_mergedにあるuser_idのみをmerge対象にしたいので、左結合(df_actions_merged.user_idを基準にマージ)する

In [22]:
df_users_dropped = df_users[["id", "name"]]
df_merged = df_actions_merged.merge(
    df_users_dropped.rename(columns={"id": "user_id"}),
    left_on=["user_id"],
    right_on=["user_id"],
    how="left")
df_merged

,action_id,user_id,date,message,point,reaction_id,reaction_user_id,reaction_date,type,reward,name
0,7GsubTX9n6,BcHgeZkTsc,2021-08-15T10:12:34+09:00,Hoge hoge,1,ycV0zbqrL5,AQA7LkexXv,2021-08-15T11:52:12+09:00,like,1,アリス
1,D76FJVQ2j2,KHPiabVr3o,2021-08-15T10:23:45+09:00,Lorem Ipsum,21,jwUcUCuVEq,BcHgeZkTsc,2021-08-15T16:21:48+09:00,comment,3,ボブ
2,6znyhCukd6,HzZow64HGH,2021-08-15T11:54:32+09:00,テスト テスト,3,NaN,NaN,NaN,NaN,0,エリザベス
3,hSQszmDjlU,BcHgeZkTsc,2021-08-15T12:34:56+09:00,テスト\nテスト２,42,NaN,NaN,NaN,NaN,0,アリス
4,CVqQD0xH2Y,HzZow64HGH,2021-08-15T14:36:52+09:00,👑👑💢,49,NaN,NaN,NaN,NaN,0,エリザベス
5,OoXNK4b3px,BcHgeZkTsc,2021-08-15T14:41:03+09:00,🔡,0,NXXk7iEsMA,AQA7LkexXv,2021-08-15T13:09:31+09:00,like,1,アリス
6,OoXNK4b3px,BcHgeZkTsc,2021-08-15T14:41:03+09:00,🔡,0,Oq9i6DBTBp,AQA7LkexXv,2021-08-15T13:21:47+09:00,like,1,アリス
7,veDQHBOXnG,BcHgeZkTsc,2021-08-15T14:52:12+09:00,foo,4,NaN,NaN,NaN,NaN,0,アリス


## ユーザごとの活動量(point + reward)を計算

rowはuser_idごとのDataFrameが入る

そのため、row.point + row.rewardはSeriesになる。

- サンプルコード
```python
df_merged \
    .groupby("user_id") \
    .apply(lambda row: row.point + row.reward)
```

- サンプルコードの実行結果
```shell
user_id      
BcHgeZkTsc  0     2
            3    42
            5     1
            6     1
            7     4
HzZow64HGH  2     3
            4    49
KHPiabVr3o  1    24
dtype: int64
```

本例ではuser_idごとの活動量を出したいので、Seriesをsumする必要がある。

In [23]:
df_sum = df_merged \
    .groupby("user_id", as_index=False) \
    .apply(lambda row: sum(row.point + row.reward))
df_sum.columns = df_sum.columns.fillna('activity')
df_sum

,user_id,activity
0,BcHgeZkTsc,50
1,HzZow64HGH,52
2,KHPiabVr3o,24


## ユーザごとに活動量のランキングを出す

In [24]:
df_ranking = df_sum \
    .merge(
        df_users_dropped.rename(columns={"id": "user_id"}),
        on="user_id",
        how="left"
    ) \
    .sort_values(["activity"], ascending=False) \
    .reset_index(drop=True)
df_ranking

,user_id,activity,name
0,HzZow64HGH,52,エリザベス
1,BcHgeZkTsc,50,アリス
2,KHPiabVr3o,24,ボブ
